In [1]:
%cd "../"
%pwd

import numpy as np
import matplotlib.pyplot as plt

import io 
import pandas as pd 
import pyreadr

from load_data import *
from overlap_genes import *

C:\Users\alexi\Desktop\Memory_genes\src


In [2]:
#Load AE3 data
AE3 = pd.read_csv ('../data/merged_data/AE3.csv')
AE3 = AE3.set_index('Unnamed: 0')
y_AE3 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_AE3.csv')))
print(np.shape(AE3), np.shape(y_AE3))

#Load AE4 data
AE4 = pd.read_csv ('../data/merged_data/AE4.csv')
AE4 = AE4.set_index('Unnamed: 0')
y_AE4 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_AE4.csv')))
print(np.shape(AE4), np.shape(y_AE4))

#Load BIDDY D0 data
D0 = pd.read_csv ('../data/merged_data/D0.csv')
D0 = D0.set_index('Unnamed: 0')
y_D0 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_D0.csv')))
print(np.shape(D0), np.shape(y_D0))

#Load BIDDY D6 data
D6 = pd.read_csv ('../data/merged_data/D6.csv')
D6 = D6.set_index('Unnamed: 0')
y_D6 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_D6.csv')))
print(np.shape(D6), np.shape(y_D6))

#Load BIDDY D15 data
D15 = pd.read_csv ('../data/merged_data/D15.csv')
D15 = D15.set_index('Unnamed: 0')
y_D15 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_D15.csv')))
print(np.shape(D15), np.shape(y_D15))

#Load Weinreb LK data
LK = pd.read_csv ('../data/merged_data/LK.csv')
LK = LK.set_index('Unnamed: 0')
y_LK  = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_LK.csv')))
print(np.shape(LK), np.shape(y_LK))

#Load Weinreb LSK data
LSK = pd.read_csv ('../data/merged_data/LSK.csv')
LSK = LSK.set_index('Unnamed: 0')
y_LSK  =np.squeeze(np.array(pd.read_csv('../data/merged_data/y_LSK.csv')))
print(np.shape(LSK), np.shape(y_LSK))

#Load Weinreb LSKmix data
LSKmix = pd.read_csv ('../data/merged_data/LSKmix.csv')
LSKmix = LSKmix.set_index('Unnamed: 0')
y_LSKmix  = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_LSKmix.csv')))
print(np.shape(LSKmix), np.shape(y_LSKmix))

(32285, 333) (333,)
(32285, 1473) (1473,)
(32285, 461) (461,)
(32285, 154) (154,)
(32285, 178) (178,)
(16760, 610) (610,)
(25289, 3383) (3383,)
(23461, 1023) (1023,)


# Fuse all data sets

In [3]:
genes_interest = set(AE3.index) & set(LK.index)
genes_interest = genes_interest & set(LSKmix.index)

AE3_df = AE3.loc[genes_interest,:]
AE4_df = AE4.loc[genes_interest,:]
D0_df = D0.loc[genes_interest,:]
D6_df = D6.loc[genes_interest,:]
D15_df = D15.loc[genes_interest,:]
LK_df = LK.loc[genes_interest,:]
LSK_df = LSK.loc[genes_interest,:]
LSKmix_df = LSKmix.loc[genes_interest,:]

In [4]:
#Get name of genes that are repeated in Weinreb data
genes, counts_gene = np.unique(LK_df.index, return_counts=True)
ind = np.where(counts_gene>1) #get indices of genes appearing more than once
non_unique_genes = genes[ind]

genes, counts_gene = np.unique(LSK_df.index, return_counts=True)
ind = np.where(counts_gene>1) #get indices of genes appearing more than once
non_unique_genes = np.append(non_unique_genes, genes[ind])

genes, counts_gene = np.unique(LSKmix_df.index, return_counts=True)
ind = np.where(counts_gene>1) #get indices of genes appearing more than once
non_unique_genes = np.append(non_unique_genes, genes[ind])

non_unique_genes= list(np.unique(non_unique_genes))

#Remove this genes from all data set
AE3_df = AE3_df.drop(index= non_unique_genes)
AE4_df = AE4_df.drop(non_unique_genes)
D0_df = D0_df.drop(non_unique_genes)
D6_df = D6_df.drop(non_unique_genes)
D15_df = D15_df.drop(non_unique_genes)
LK_df = LK_df.drop(non_unique_genes)
LSK_df = LSK_df.drop(non_unique_genes)
LSKmix_df = LSKmix_df.drop(non_unique_genes)

In [5]:
len(non_unique_genes)

184

In [6]:
pd.DataFrame(non_unique_genes).to_csv('../data/non_unique_genes.csv', index=False)

In [20]:
#Fuse ormalized data
all_norm = pd.concat([AE3_df, AE4_df,D0_df,D6_df,D15_df,LK_df,LSK_df,LSKmix_df], axis=1)
print(all_norm.shape)

#Fuse families info
y_AE4 += max(y_AE3) #Add the max indice of families in first dataset so that there is no overlap between families indices.
y_all = np.hstack((y_AE3, y_AE4))

y_D0 += max(y_all)
y_all = np.hstack((y_all, y_D0))

y_D6 += max(y_all)
y_all = np.hstack((y_all, y_D6))

y_D15 += max(y_all)
y_all = np.hstack((y_all, y_D15))

y_LK += max(y_all)
y_all = np.hstack((y_all, y_LK))

y_LSK += max(y_all)
y_all = np.hstack((y_all, y_LSK)) 

y_LSKmix += max(y_all)
y_all = np.hstack((y_all, y_LSKmix))
print(len(y_all))

(16051, 7615)
7615


In [24]:
#Remove all genes that are not expressed in at least percentage (default 50%) of the all cells
gene_expressed = filter_norm_data(all_norm,0.05)
all_norm = all_norm[gene_expressed]
print(all_norm.shape)

#Store the name of genes of interest
genes_interest = all_norm.index.values #get name of genes
pd.DataFrame(genes_interest).to_csv('../data/processed_data/allgenes_interest.csv', index=False)

#Create preprocess data 
all_norm.to_csv('../data/processed_data/all.csv', index=False)
pd.DataFrame(y_all).to_csv('../data/processed_data/y_all.csv', index=False)

(9353, 7615)


In [25]:
#Add .csv file that remembers which cells from which data set -> give indices
sizes = [np.shape(y_AE3), np.shape(y_AE4), np.shape(y_D0), np.shape(y_D6), np.shape(y_D15), np.shape(y_LK), np.shape(y_LSK), np.shape(y_LSKmix)]
pd.DataFrame(sizes).to_csv('../data/processed_data/size_datasets.csv', index=False)